In [72]:
def parsePicksBans(inputData):
    '''
    Returns 4 lists in this order: RadianPicks, RadiantBans, DirePicks, DireBans
    '''

    if inputData is None:
        return None, None, None, None
    
    parsed_data = json.loads(inputData)

    picks_rad = [item["hero_id"] for item in parsed_data if item["is_pick"] and item["team"]]
    picks_dire = [item["hero_id"] for item in parsed_data if item["is_pick"] and not item["team"]]

    bans_rad = [item["hero_id"] for item in parsed_data if not item["is_pick"] and item["team"]]
    bans_dire = [item["hero_id"] for item in parsed_data if not item["is_pick"] and not item["team"]]

    return picks_rad, bans_rad, picks_dire, bans_dire


def selectDataFromDBToDF(select_query):
    # Execute the query
    cursor.execute(select_query)
    # Fetch all the rows from the result set
    rows = cursor.fetchall()
    # Get the column names from the cursor description
    columns = [desc[0] for desc in cursor.description]
    # Create a Pandas DataFrame from the fetched data
    df = pd.DataFrame(rows, columns=columns)
    return df

def countPicksBans(dataFrame, seperateTeams = False):
    if seperateTeams:
        return output
    else:

        # count picks
        object_list = [item for sublist in zip(dataFrame['picks_rad'], dataFrame['picks_dire']) for item in sublist]
        object_win_list = [item for sublist in zip(dataFrame[dataFrame['radiant_win']==1]['picks_rad'], dataFrame[dataFrame['radiant_win']==0]['picks_dire']) for item in sublist]
        
        cnt = Counter()
        cnt2 = Counter()

        for obj in object_list:
            cnt.update(obj)

        for obj in object_win_list:
            cnt2.update(obj)

        picks = counterToTable(cnt)
        picks_win = counterToTable(cnt2)
        picks['percentage'] = picks['count'] / len(dataFrame)

        picks = pd.merge(picks, picks_win[['hero_id', 'count']], on='hero_id', suffixes=('','_win'))
        picks['win_%']=picks['count_win']/picks['count']


        # count bans
        object_list = [item for sublist in zip(dataFrame['bans_rad'], dataFrame['bans_dire']) for item in sublist]
        object_win_list = [item for sublist in zip(dataFrame[dataFrame['radiant_win']==1]['bans_rad'], dataFrame[dataFrame['radiant_win']==0]['bans_dire']) for item in sublist]
        
        cnt = Counter()
        cnt2 = Counter()

        for obj in object_list:
            cnt.update(obj)

        for obj in object_win_list:
            cnt2.update(obj)

        bans = counterToTable(cnt)
        bans_win = counterToTable(cnt2)
        bans['percentage'] = bans['count'] / len(dataFrame)

        bans = pd.merge(bans, bans_win[['hero_id', 'count']], on='hero_id', suffixes=('','_win'))
        bans['win_%']=bans['count_win']/bans['count']

        return picks, bans
    
def counterToTable(counter):
    table = pd.DataFrame.from_dict(counter, orient="index").reset_index()
    table.rename(columns={'index':'hero_id', 0 :'count'}, inplace=True)
    table = pd.merge(table, heros[['hero_id', 'localized_name']], on="hero_id", how="left")    
    table = table.sort_values('count', ascending=False)
    return table

In [85]:
import mysql.connector
import pandas as pd
import json
from collections import Counter

db = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "root",
    database = "dota2"
    )

cursor = db.cursor()

table_name = 'match_data'

# Define your SQL query to select data
select_query = "SELECT picks_bans, leagueid, radiant_gold_adv, radiant_xp_adv, radiant_win FROM match_data where leagueid ='16169'"

# Execute the query
cursor.execute(select_query)

# Fetch all the rows from the result set
rows = cursor.fetchall()

# Get the column names from the cursor description
columns = [desc[0] for desc in cursor.description]

# Create a Pandas DataFrame from the fetched data
df = pd.DataFrame(rows, columns=columns)
df.head()

heros = selectDataFromDBToDF("SELECT * FROM hero_data")
heros.rename(columns={'id':'hero_id'}, inplace=True)

# parse picks and bans
df[["picks_rad", "bans_rad", "picks_dire", "bans_dire"]]  = df['picks_bans'].apply(parsePicksBans).apply(pd.Series)
# count pick and bans
#picks, bans = countPicksBans(df)


In [86]:

df


,picks_bans,leagueid,radiant_gold_adv,radiant_xp_adv,radiant_win,picks_rad,bans_rad,picks_dire,bans_dire
0,None,16169,None,None,1,None,None,None,None
1,None,16169,None,None,0,None,None,None,None
2,None,16169,None,None,0,None,None,None,None
3,None,16169,None,None,1,None,None,None,None
4,None,16169,None,None,1,None,None,None,None
...,...,...,...,...,...,...,...,...,...
88,None,16169,None,None,0,None,None,None,None
89,None,16169,None,None,0,None,None,None,None
90,None,16169,None,None,1,None,None,None,None
91,None,16169,None,None,0,None,None,None,None


In [77]:
bans.head(5)

,hero_id,count,localized_name,percentage,count_win,win_%
0,98,133,Timbersaw,0.707447,62,0.466165
1,66,116,Chen,0.617021,56,0.482759
2,69,115,Doom,0.611702,46,0.400000
3,41,96,Faceless Void,0.510638,43,0.447917
4,53,92,Nature's Prophet,0.489362,51,0.554348


In [82]:
picks.sort_values(['win_%', 'count'], ascending=True).head(20)

,hero_id,count,localized_name,percentage,count_win,win_%
59,93,12,Slark,0.063830,2,0.166667
30,66,21,Chen,0.111702,4,0.190476
80,101,5,Skywrath Mage,0.026596,1,0.200000
43,123,18,Hoodwink,0.095745,4,0.222222
66,85,8,Undying,0.042553,2,0.250000
47,54,15,Lifestealer,0.079787,4,0.266667
74,22,7,Zeus,0.037234,2,0.285714
23,90,28,Keeper of the Light,0.148936,8,0.285714
5,96,47,Centaur Warrunner,0.250000,14,0.297872
46,38,16,Beastmaster,0.085106,5,0.312500
